In [3]:
import time
import numpy as np
from mss import mss
import win32gui

def monitor_health(window_title, x_start, bar_len, y, fps, lower_bgr, upper_bgr, drop_per_px, reverse=False):
    """
    Live health bar monitor.
    
    Parameters:
        window_title (str): Title of the window to capture
        x_start (int): X offset of the bar's left edge in client coordinates
        bar_len (int): Length of the bar in pixels
        y (int): Y offset of the bar in client coordinates
        fps (int): Frames per second to capture
        lower_bgr (np.ndarray): Lower BGR threshold array [B, G, R]
        upper_bgr (np.ndarray): Upper BGR threshold array [B, G, R]
        drop_per_px (float): Percentage drop per pixel (e.g., 0.25 for ¼%)
        reverse (bool): If True, bar depletes from right to left (for Player 2)
    """
    # Locate the window
    hwnd = win32gui.FindWindow(None, window_title)
    if not hwnd:
        raise RuntimeError(f"Window not found: {window_title!r}")
    
    left, top = win32gui.ClientToScreen(hwnd, (0, 0))
    roi = {
        'left': left + x_start,
        'top': top + y,
        'width': bar_len,
        'height': 1,
    }
    
    sct = mss()
    interval = 1.0 / fps
    player = "Player 2" if reverse else "Player 1"
    
    print(f"Starting live health monitor for {player} (Ctrl-C to stop)\n")
    
    try:
        while True:
            t0 = time.perf_counter()
            
            # Capture the bar strip and split channels
            raw = sct.grab(roi)
            strip = np.array(raw)[:, :, :3]  # shape=(1,bar_len,3)
            b, g, r = strip[0].T  # each length=bar_len
            
            # Build a mask of 'yellow' pixels
            mask = (
                (r >= lower_bgr[2]) & (r <= upper_bgr[2]) &
                (g >= lower_bgr[1]) & (g <= upper_bgr[1]) &
                (b >= lower_bgr[0]) & (b <= upper_bgr[0])
            )
            
            # Find last non-yellow index based on direction
            non_yellow = np.nonzero(~mask)[0]
            
            if reverse:  # Player 2: bar depletes from right to left
                # For Player 2, we want the first non-yellow pixel from the left
                last_idx = non_yellow.min() if non_yellow.size else bar_len
                drop_pixels = bar_len - last_idx
            else:  # Player 1: bar depletes from left to right
                # For Player 1, we want the last non-yellow pixel from the left
                last_idx = non_yellow.max() if non_yellow.size else -1
                drop_pixels = max(0, last_idx + 1)
            
            # Compute life percentage
            life_pct = 100.0 - (drop_pixels * drop_per_px)
            life_pct = np.clip(life_pct, 0.0, 100.0)
            
            # Print results
            print(f"{player}:")
            if (not reverse and last_idx < 0) or (reverse and last_idx >= bar_len):
                print("  No non-yellow pixel detected → life = 100.00%\n")
            else:
                x_coord = x_start + last_idx
                print(f"  Last non-yellow at x = {x_coord}")
                for rel in (-1, 0, +1):
                    idx = last_idx + rel
                    if 0 <= idx < bar_len:
                        R, G, B = int(r[idx]), int(g[idx]), int(b[idx])
                        print(f"    rel {rel:+} (x={x_start+idx}): RGB=({R:3d},{G:3d},{B:3d})")
                print(f"  Computed life: {life_pct:6.2f}%\n")
            
            # Throttle to target FPS
            dt = time.perf_counter() - t0
            if dt < interval:
                time.sleep(interval - dt)
                
    except KeyboardInterrupt:
        print(f"\n{player} monitor stopped.")

def monitor_both_players(window_title, p1_x, p2_x, bar_len, y, fps, lower_bgr, upper_bgr, drop_per_px):
    """Monitor both players simultaneously using threading."""
    import threading
    
    # Create threads for each player
    p1_thread = threading.Thread(
        target=monitor_health,
        args=(window_title, p1_x, bar_len, y, fps, lower_bgr, upper_bgr, drop_per_px, False)
    )
    
    p2_thread = threading.Thread(
        target=monitor_health,
        args=(window_title, p2_x - bar_len, bar_len, y, fps, lower_bgr, upper_bgr, drop_per_px, True)
    )
    
    # Start both threads
    p1_thread.start()
    p2_thread.start()
    
    # Wait for both to complete
    p1_thread.join()
    p2_thread.join()

if __name__ == "__main__":
    # Default configuration
    WINDOW_TITLE = "Bloody Roar II (USA) [PlayStation] - BizHawk"
    PLAYER_1_X = 500      # P1 bar starts at 505
    PLAYER_2_X = 1421     # P2 bar starts at 1421 (but we'll calculate from 1021)
    BAR_LEN = 400
    Y = 155
    FPS = 30
    LOWER_BGR = np.array([0, 160, 190], dtype=np.uint8)
    UPPER_BGR = np.array([20, 180, 220], dtype=np.uint8)
    DROP_PER_PX = 0.25
    
    # Monitor both players simultaneously
    # monitor_both_players(
    #     WINDOW_TITLE,
    #     PLAYER_1_X,
    #     PLAYER_2_X,
    #     BAR_LEN,
    #     Y,
    #     FPS,
    #     LOWER_BGR,
    #     UPPER_BGR,
    #     DROP_PER_PX
    # )
    
    # Or monitor individually:
    # Player 1 only:
    monitor_health(WINDOW_TITLE, PLAYER_1_X, BAR_LEN, Y, FPS, LOWER_BGR, UPPER_BGR, DROP_PER_PX, False)
    
    # Player 2 only:
    # monitor_health(WINDOW_TITLE, 1021, BAR_LEN, Y, FPS, LOWER_BGR, UPPER_BGR, DROP_PER_PX, True)

Starting live health monitor for Player 1 (Ctrl-C to stop)

Player 1:
  Last non-yellow at x = 525
    rel -1 (x=524): RGB=( 89,109,134)
    rel +0 (x=525): RGB=(169,153, 48)
    rel +1 (x=526): RGB=(206,172, 12)
  Computed life:  93.50%

Player 1:
  Last non-yellow at x = 525
    rel -1 (x=524): RGB=( 89,109,134)
    rel +0 (x=525): RGB=(169,153, 48)
    rel +1 (x=526): RGB=(206,172, 12)
  Computed life:  93.50%

Player 1:
  Last non-yellow at x = 525
    rel -1 (x=524): RGB=( 89,109,134)
    rel +0 (x=525): RGB=(169,153, 48)
    rel +1 (x=526): RGB=(206,172, 12)
  Computed life:  93.50%

Player 1:
  Last non-yellow at x = 525
    rel -1 (x=524): RGB=( 89,109,134)
    rel +0 (x=525): RGB=(169,153, 48)
    rel +1 (x=526): RGB=(206,172, 12)
  Computed life:  93.50%

Player 1:
  Last non-yellow at x = 525
    rel -1 (x=524): RGB=( 89,109,134)
    rel +0 (x=525): RGB=(169,153, 48)
    rel +1 (x=526): RGB=(206,172, 12)
  Computed life:  93.50%

Player 1:
  Last non-yellow at x = 525
    re